In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
# Define the objective function for the portfolio optimization
def objective(x,mean_returns,cov_matrix):
    port_returns = np.sum(mean_returns * x)
    port_volatility = np.sqrt(np.dot(x.T, np.dot(cov_matrix, x)))
    penalty = 10 * np.abs(np.sum(x) - 1) # penalty for sum of portfolio weights not equal to 1
    return -port_returns/port_volatility + penalty

In [ ]:
def fa_algorithm(n_solutions, n_iter, lb, ub, objective_func,mean_returns,cov_matrix, alpha=0.5, beta=1, gamma=1):
    n_dim = len(lb)
    best_sol = None
    best_obj = np.inf
    solutions = np.random.dirichlet(np.ones(len(lb)), size=n_solutions) * (ub-lb) + lb
    solutions = solutions / solutions.sum(axis=1, keepdims=True)
    for i in range(n_iter):
        attractiveness = np.zeros(n_solutions)
        for j in range(n_solutions):
            for k in range(n_solutions):
                if objective_func(solutions[k],mean_returns,cov_matrix) < objective_func(solutions[j],mean_returns,cov_matrix):
                    attractiveness[j] += (1 / (1 + np.linalg.norm(solutions[j] - solutions[k])))
            attractiveness[j] *= np.random.rand()
        index = np.argsort(attractiveness)[::-1]
        best_firefly = solutions[index[0]]
        for j in range(n_solutions):
            beta0 = 1
            beta = beta0 * np.exp(-0.2 * j)
            r = np.linalg.norm(solutions[j] - best_firefly)
            new_sol = solutions[j] + beta * (best_firefly - solutions[j]) + 0.01 * np.random.randn(n_dim)
            new_sol = np.maximum(new_sol, lb)
            new_sol = np.minimum(new_sol, ub)
            new_obj = objective_func(new_sol,mean_returns,cov_matrix)
            if new_obj < objective_func(solutions[j],mean_returns,cov_matrix):
                solutions[j] = new_sol
            if new_obj < best_obj:
                best_sol = new_sol
                best_obj = new_obj
    return best_sol, best_obj


In [ ]:
# Define the hybrid algorithm
def hybrid_algorithm(n_solutions, n_iter, lb, ub, objective_func, alpha, beta0, gamma,mean_returns,cov_matrix):
    fa_solutions, fa_obj = fa_algorithm(n_solutions, n_iter, lb, ub, objective_func,mean_returns,cov_matrix)
    return fa_solutions, fa_obj


In [ ]:
returns_df = pd.read_csv("/content/rf_test_returns.csv")

In [ ]:
returns_df

,PFE,JNJ,GOOGL,AAPL,COST,WMT,KR,JPM,BAC,HSBC
0,-0.000994,0.001739,-0.000057,-0.000408,-0.059435,0.000260,-0.001322,-0.002213,-0.002041,-0.001925
1,-0.023281,0.001019,0.000148,-0.003555,-0.019503,-0.001084,-0.027990,0.000377,-0.014027,-0.000206
2,-0.040311,-0.012427,0.000546,-0.001842,-0.057652,0.000713,-0.108343,-0.003936,-0.043978,0.000144
3,-0.037932,-0.019114,0.002493,-0.012019,-0.012637,-0.003582,-0.070462,-0.003723,-0.116553,-0.014434
4,-0.013313,-0.011961,0.001973,-0.003728,-0.003525,-0.001574,-0.102715,-0.015542,-0.024431,0.000517
...,...,...,...,...,...,...,...,...,...,...
199,-0.000387,0.004847,0.039658,-0.012989,-0.007535,0.027911,-0.004318,-0.002144,0.003303,0.005593
200,-0.001020,0.149740,0.005037,0.038903,0.008831,0.014800,0.022259,0.000025,0.000025,0.035836
201,-0.002341,-0.051964,-0.094449,0.017800,0.000960,-0.005098,0.006711,-0.000559,-0.010822,-0.010754
202,0.004906,0.037063,0.099094,0.006095,0.080428,0.007396,0.001948,0.001784,0.012721,0.017379


In [ ]:
roi=[]
s_r=[]
assets = ["PFE", "JNJ", "GOOGL", "AAPL", "COST", "WMT", "KR", "JPM",
          "BAC", "HSBC"]

In [ ]:
n_solutions = 100
n_iter = 50
alpha = 0.1
beta0 = 1
gamma = 2

lb = np.array([0] * len(assets))
ub = np.array([1] * len(assets))

In [ ]:
lb

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
ub

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
returns_df

,PFE,JNJ,GOOGL,AAPL,COST,WMT,KR,JPM,BAC,HSBC
0,-0.000994,0.001739,-0.000057,-0.000408,-0.059435,0.000260,-0.001322,-0.002213,-0.002041,-0.001925
1,-0.023281,0.001019,0.000148,-0.003555,-0.019503,-0.001084,-0.027990,0.000377,-0.014027,-0.000206
2,-0.040311,-0.012427,0.000546,-0.001842,-0.057652,0.000713,-0.108343,-0.003936,-0.043978,0.000144
3,-0.037932,-0.019114,0.002493,-0.012019,-0.012637,-0.003582,-0.070462,-0.003723,-0.116553,-0.014434
4,-0.013313,-0.011961,0.001973,-0.003728,-0.003525,-0.001574,-0.102715,-0.015542,-0.024431,0.000517
...,...,...,...,...,...,...,...,...,...,...
199,-0.000387,0.004847,0.039658,-0.012989,-0.007535,0.027911,-0.004318,-0.002144,0.003303,0.005593
200,-0.001020,0.149740,0.005037,0.038903,0.008831,0.014800,0.022259,0.000025,0.000025,0.035836
201,-0.002341,-0.051964,-0.094449,0.017800,0.000960,-0.005098,0.006711,-0.000559,-0.010822,-0.010754
202,0.004906,0.037063,0.099094,0.006095,0.080428,0.007396,0.001948,0.001784,0.012721,0.017379


In [ ]:
returns_df.shape[0]

204

In [ ]:
def opt_portfolio(i,mean_returns,cov_matrix):
    optimized_portfolio = hybrid_algorithm(n_solutions, n_iter, lb, ub, objective, alpha, beta0, gamma,mean_returns,cov_matrix)
    print("Portfolio allocation:", optimized_portfolio[0])
    portfolio_returns = np.sum(optimized_portfolio[0] * mean_returns)
    portfolio_volatility = np.sqrt(np.dot(optimized_portfolio[0].T, np.dot(cov_matrix,optimized_portfolio[0])))
    sharpe_ratio = portfolio_returns / portfolio_volatility
    sortino_ratio = portfolio_returns / np.sqrt(np.mean(np.minimum(returns_df, 0)**2))
    max_drawdown = np.max(np.maximum.accumulate((1+returns_df).cumprod()) - (1+returns_df).cumprod())
    print("Results for day: ",i)
    print("Portfolio allocation:\n",optimized_portfolio[0])
    print("Sharpe ratio:\n", sharpe_ratio)
    print("Sortino ratio:\n", sortino_ratio)
    print("Max drawdown:\n", max_drawdown)
    return(optimized_portfolio[0])

In [ ]:
def add(L):
  res = list()
  for j in range(0, len(L[0])):
    tmp = 0
    for i in range(0, len(L)):
        tmp = tmp + L[i][j]
    res.append(tmp)
  arr = np.array(res)
  result = arr/len(L[0])
  res = result.tolist()
  return(res)

In [ ]:
def calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov):
  cumulative_returns = 0
  cumulative_volatilities = 0

  for i in range(len(normalized_opt)):
      #print(i)
      # Calculate portfolio returns

      portfolio_returns = np.sum(normalized_opt[i] * interval_returns[i])
      #print(portfolio_returns)
      # Calculate portfolio volatility
      portfolio_volatility = np.sqrt(np.dot(normalized_opt[i].T, np.dot(interval_cov[i], normalized_opt[i])))
      #print(portfolio_volatility)
      # Add to cumulative returns and volatilities
      cumulative_returns += portfolio_returns
      cumulative_volatilities += portfolio_volatility

  sharpe_ratio = cumulative_returns / cumulative_volatilities
  print("Sharpe Ratio:", sharpe_ratio)
  return(sharpe_ratio)


In [ ]:
def calculate_roi(normalized_opt,ditv):
  i=0
  k=1
  capital=10000
  final_investment=[]
  final_amount=0
  invest = np.array([capital] * len(assets))
  investment = [x * y for x, y in zip(invest, normalized_opt[0])]
  remain=10000-sum(investment)
  for j in range(1,151):
    row_list = returns_df.loc[j, :].values.flatten().tolist()
    investment = [(x+(x * y)) for x, y in zip(investment, row_list)]
    i=i+1
    if(i==ditv and j!=150 ):
      print(investment)
      amount=sum(investment)+remain
      print("Investment at day ",j," is ",amount)
      cap = sum(investment)+remain
      investment = np.array([cap] * len(assets))
      investment = [x * y for x, y in zip(investment, normalized_opt[k])]
      print("New portfolio investment : ",investment)
      temp=sum(investment)
      print("Investment amount ",j," is ",amount)
      remain=cap-sum(investment)
      print("Remaning amount",remain)
      k=k+1
      i=0
    elif(j==150):
      print("Final Portfolio investment")
      final_investment=investment.copy()
      print(investment)
      amount=sum(investment)+remain
      final_amount=amount
      print("Investment at day ",j," is ",amount)

  n_periods = 252 / 150
  roi = ((final_amount / capital) - 1) * 100
  annualized_roi = (((1 + roi / 100) ** n_periods) - 1)*100
  print("Annualized ROI:", annualized_roi , "%")
  return(annualized_roi)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==5):
    temp_df=returns_df.iloc[j-4:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt",len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,5))

        PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
1 -0.023281  0.001019  0.000148 -0.003555 -0.019503 -0.001084 -0.027990   
2 -0.040311 -0.012427  0.000546 -0.001842 -0.057652  0.000713 -0.108343   
3 -0.037932 -0.019114  0.002493 -0.012019 -0.012637 -0.003582 -0.070462   
4 -0.013313 -0.011961  0.001973 -0.003728 -0.003525 -0.001574 -0.102715   
5 -0.005140  0.012929 -0.005124  0.001886  0.055126  0.001520  0.007393   

        JPM       BAC      HSBC  
1  0.000377 -0.014027 -0.000206  
2 -0.003936 -0.043978  0.000144  
3 -0.003723 -0.116553 -0.014434  
4 -0.015542 -0.024431  0.000517  
5  0.006758  0.005621 -0.000272  
            PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
PFE    0.000233  0.000146 -0.000030  0.000048  0.000526  0.000012  0.000443   
JNJ    0.000146  0.000164 -0.000036  0.000051  0.000367  0.000018  0.000551   
GOOGL -0.000030 -0.000036  0.000009 -0.000012 -0.000089 -0.000005 -0.000119   
AAPL   0.000048  0.000051 -0.

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.20814772 0.03128879 0.03209959 0.23444842 0.13783303 0.02732746
 0.01349991 0.01711265 0.29800791 0.        ]
Results for day:  5
Portfolio allocation:
 [0.20814772 0.03128879 0.03209959 0.23444842 0.13783303 0.02732746
 0.01349991 0.01711265 0.29800791 0.        ]
Sharpe ratio:
 0.2626713681030797
Sortino ratio:
 PFE      0.482412
JNJ      0.333997
GOOGL    0.340939
AAPL     0.365992
COST     0.262900
WMT      0.329919
KR       0.173711
JPM      0.263677
BAC      0.215990
HSBC     0.278042
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
11  0.007194 -0.000169 -0.002893  0.011659  0.058368  0.000025  0.012735   
12 -0.004374  0.001128  0.000947 -0.003822 -0.002371 -0.000191 -0.006676   
13 -0.022181  0.001385  0.

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.02801098 0.00747656 0.54306525 0.06506066 0.05902033 0.05756723
 0.         0.         0.21275379 0.02750239]
Results for day:  5
Portfolio allocation:
 [0.02801098 0.00747656 0.54306525 0.06506066 0.05902033 0.05756723
 0.         0.         0.21275379 0.02750239]
Sharpe ratio:
 0.11311773078051221
Sortino ratio:
 PFE      0.063256
JNJ      0.043795
GOOGL    0.044705
AAPL     0.047990
COST     0.034472
WMT      0.043260
KR       0.022778
JPM      0.034574
BAC      0.028321
HSBC     0.036458
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
16  0.043882 -0.000276  0.001157  0.005644 -0.002034  0.002470  0.092866   
17  0.000333 -0.000568 -0.002004  0.004139 -0.053446  0.000009 -0.006565   
18 -0.008957 -0.002120  0

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.06052428 0.2521918  0.04760961 0.02475738 0.05529664 0.08973773
 0.00991363 0.29288325 0.         0.16827231]
Results for day:  5
Portfolio allocation:
 [0.06052428 0.2521918  0.04760961 0.02475738 0.05529664 0.08973773
 0.00991363 0.29288325 0.         0.16827231]
Sharpe ratio:
 1.0494843940226857
Sortino ratio:
 PFE      0.189551
JNJ      0.131235
GOOGL    0.133963
AAPL     0.143807
COST     0.103300
WMT      0.129633
KR       0.068255
JPM      0.103605
BAC      0.084868
HSBC     0.109249
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
146  0.029045 -0.010665  0.005606 -0.000395  0.035673  0.006717  0.071136   
147  0.039256 -0.017967 -0.004508 -0.001617  0.001261  0.000590 -0.055194   
148 -0.000063  0.009648

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.14791788 0.23989404 0.02923066 0.07925795 0.24106841 0.13355743
 0.03779161 0.03235978 0.         0.0535864 ]
Results for day:  5
Portfolio allocation:
 [0.14791788 0.23989404 0.02923066 0.07925795 0.24106841 0.13355743
 0.03779161 0.03235978 0.         0.0535864 ]
Sharpe ratio:
 0.6552140322959338
Sortino ratio:
 PFE      0.326106
JNJ      0.225778
GOOGL    0.230471
AAPL     0.247407
COST     0.177718
WMT      0.223022
KR       0.117427
JPM      0.178243
BAC      0.146007
HSBC     0.187953
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
Length of opt 30
[[0.025 0.103 0.041 0.108 0.332 0.082 0.074 0.151 0.003 0.071]
 [0.207 0.03  0.031 0.234 0.137 0.026 0.013 0.016 0.297 0.   ]
 [0.027 0.006 0.542 0.064 0.058 0.057 0.    0.    0.212 0.026]
 [0.262 0.112 0.197 0.075 0.03  0.103 0.014 0.08

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==10):
    temp_df=returns_df.iloc[j-9:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt", len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,10))

         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
1  -0.023281  0.001019  0.000148 -0.003555 -0.019503 -0.001084 -0.027990   
2  -0.040311 -0.012427  0.000546 -0.001842 -0.057652  0.000713 -0.108343   
3  -0.037932 -0.019114  0.002493 -0.012019 -0.012637 -0.003582 -0.070462   
4  -0.013313 -0.011961  0.001973 -0.003728 -0.003525 -0.001574 -0.102715   
5  -0.005140  0.012929 -0.005124  0.001886  0.055126  0.001520  0.007393   
6  -0.044995 -0.004363  0.003789 -0.002337 -0.004410  0.000260 -0.076160   
7   0.037793 -0.003432 -0.002888 -0.001089 -0.037880 -0.000594 -0.022043   
8   0.008316 -0.000478 -0.004688  0.004130  0.128447  0.002887  0.101177   
9  -0.000986 -0.000160 -0.004903  0.000855 -0.013351 -0.000011  0.002638   
10  0.053411 -0.004477  0.002303  0.012440 -0.033731  0.000418 -0.002852   

         JPM       BAC      HSBC  
1   0.000377 -0.014027 -0.000206  
2  -0.003936 -0.043978  0.000144  
3  -0.003723 -0.116553 -0.014434  
4  -0.015542 -0.024431 

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.08271257 0.12489582 0.26903123 0.00867099 0.08432548 0.08723836
 0.00794564 0.01472157 0.23863231 0.08252807]
Results for day:  10
Portfolio allocation:
 [0.08271257 0.12489582 0.26903123 0.00867099 0.08432548 0.08723836
 0.00794564 0.01472157 0.23863231 0.08252807]
Sharpe ratio:
 0.5011524711781236
Sortino ratio:
 PFE      0.388030
JNJ      0.268651
GOOGL    0.274235
AAPL     0.294386
COST     0.211464
WMT      0.265371
KR       0.139725
JPM      0.212090
BAC      0.173732
HSBC     0.223643
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
21 -0.031060 -0.005084  0.001370 -0.009574 -0.005994 -0.000030 -0.005079   
22  0.005454 -0.000391 -0.015565  0.000253 -0.016333  0.001674  0.006141   
23 -0.013079 -0.006689  0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.0901041  0.36157442 0.03976623 0.01950086 0.01013941 0.09154795
 0.00660889 0.18251629 0.10287201 0.09509991]
Results for day:  10
Portfolio allocation:
 [0.0901041  0.36157442 0.03976623 0.01950086 0.01013941 0.09154795
 0.00660889 0.18251629 0.10287201 0.09509991]
Sharpe ratio:
 0.31312434284549095
Sortino ratio:
 PFE      0.155098
JNJ      0.107381
GOOGL    0.109613
AAPL     0.117668
COST     0.084523
WMT      0.106070
KR       0.055849
JPM      0.084773
BAC      0.069442
HSBC     0.089391
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
31  0.001170  0.000356  0.003927 -0.002790  0.061539  0.002369  0.000662   
32 -0.008655  0.005250 -0.033020 -0.007685 -0.004530 -0.000144 -0.005015   
33  0.001279 -0.005752  

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.0027375  0.05257259 0.16337556 0.01111033 0.014287   0.0139297
 0.06668391 0.67229373 0.00183879 0.00224898]
Results for day:  10
Portfolio allocation:
 [0.0027375  0.05257259 0.16337556 0.01111033 0.014287   0.0139297
 0.06668391 0.67229373 0.00183879 0.00224898]
Sharpe ratio:
 0.1632454603538081
Sortino ratio:
 PFE      0.230020
JNJ      0.159253
GOOGL    0.162563
AAPL     0.174509
COST     0.125354
WMT      0.157309
KR       0.082827
JPM      0.125724
BAC      0.102987
HSBC     0.132573
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
141  0.023100 -0.006360  0.000426 -0.035451  0.002858  0.001401 -0.005493   
142  0.039124  0.009780 -0.000154 -0.035620  0.003294 -0.001088  0.005654   
143 -0.056805  0.008392 

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.18044383 0.22825686 0.         0.00115507 0.10635039 0.02802383
 0.00635754 0.09976835 0.         0.34988821]
Results for day:  10
Portfolio allocation:
 [0.18044383 0.22825686 0.         0.00115507 0.10635039 0.02802383
 0.00635754 0.09976835 0.         0.34988821]
Sharpe ratio:
 1.0593504181110451
Sortino ratio:
 PFE      0.531016
JNJ      0.367647
GOOGL    0.375288
AAPL     0.402866
COST     0.289388
WMT      0.363159
KR       0.191212
JPM      0.290243
BAC      0.237751
HSBC     0.306055
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
Length of opt 15
[[0.026 0.05  0.281 0.008 0.3   0.218 0.019 0.021 0.02  0.047]
 [0.082 0.124 0.268 0.008 0.083 0.086 0.007 0.014 0.237 0.081]
 [0.089 0.361 0.039 0.019 0.009 0.091 0.006 0.182 0.102 0.094]
 [0.086 0.139 0.041 0.036 0.434 0.076 0.028 0.1

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
mr=[]
i=0
opt=[]
interval_returns=[]
interval_cov=[]

In [ ]:
for j in range(1,151):
  row_list = returns_df.loc[j, :].values.flatten().tolist()
  mr.append(row_list)
  i=i+1
  if(i==15):
    temp_df=returns_df.iloc[j-14:j+1]
    print(temp_df)
    cov_matrix = temp_df.cov()
    print(cov_matrix)
    temp=[]
    m_returns=add(mr)
    temp=opt_portfolio(i,m_returns,cov_matrix)
    #print(temp)
    opt.append(temp)
    interval_returns.append(m_returns)
    interval_cov.append(cov_matrix)
    i=0
    mr=[]

normalized_opt = opt / np.sum(opt, axis=1, keepdims=True)
normalized_opt = np.round(normalized_opt, 3)

for i in range(normalized_opt.shape[0]):
  row = normalized_opt[i, :]
  if np.any(row > 0.002):
      row[row > 0.002] -= 0.001
print("Length of opt", len(normalized_opt))
print(normalized_opt)
row_sums = np.sum(normalized_opt, axis=1)
print("Sum of each row:")
print(row_sums)
s_r.append(calculate_sharp_ratio(normalized_opt,interval_returns,interval_cov))
roi.append(calculate_roi(normalized_opt,15))

         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
1  -0.023281  0.001019  0.000148 -0.003555 -0.019503 -0.001084 -0.027990   
2  -0.040311 -0.012427  0.000546 -0.001842 -0.057652  0.000713 -0.108343   
3  -0.037932 -0.019114  0.002493 -0.012019 -0.012637 -0.003582 -0.070462   
4  -0.013313 -0.011961  0.001973 -0.003728 -0.003525 -0.001574 -0.102715   
5  -0.005140  0.012929 -0.005124  0.001886  0.055126  0.001520  0.007393   
6  -0.044995 -0.004363  0.003789 -0.002337 -0.004410  0.000260 -0.076160   
7   0.037793 -0.003432 -0.002888 -0.001089 -0.037880 -0.000594 -0.022043   
8   0.008316 -0.000478 -0.004688  0.004130  0.128447  0.002887  0.101177   
9  -0.000986 -0.000160 -0.004903  0.000855 -0.013351 -0.000011  0.002638   
10  0.053411 -0.004477  0.002303  0.012440 -0.033731  0.000418 -0.002852   
11  0.007194 -0.000169 -0.002893  0.011659  0.058368  0.000025  0.012735   
12 -0.004374  0.001128  0.000947 -0.003822 -0.002371 -0.000191 -0.006676   
13 -0.022181

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.28655015 0.05233883 0.18804154 0.08344445 0.01680603 0.03892514
 0.02689343 0.05784301 0.07618359 0.17449127]
Results for day:  15
Portfolio allocation:
 [0.28655015 0.05233883 0.18804154 0.08344445 0.01680603 0.03892514
 0.02689343 0.05784301 0.07618359 0.17449127]
Sharpe ratio:
 0.8114022863887735
Sortino ratio:
 PFE      0.680772
JNJ      0.471330
GOOGL    0.481126
AAPL     0.516481
COST     0.371000
WMT      0.465576
KR       0.245138
JPM      0.372096
BAC      0.304801
HSBC     0.392367
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
31  0.001170  0.000356  0.003927 -0.002790  0.061539  0.002369  0.000662   
32 -0.008655  0.005250 -0.033020 -0.007685 -0.004530 -0.000144 -0.005015   
33  0.001279 -0.005752  0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.22203784 0.22104261 0.0198942  0.13835424 0.28618132 0.01665461
 0.07033953 0.         0.02535296 0.0009899 ]
Results for day:  15
Portfolio allocation:
 [0.22203784 0.22104261 0.0198942  0.13835424 0.28618132 0.01665461
 0.07033953 0.         0.02535296 0.0009899 ]
Sharpe ratio:
 0.6682224864599788
Sortino ratio:
 PFE      0.692914
JNJ      0.479736
GOOGL    0.489707
AAPL     0.525692
COST     0.377617
WMT      0.473879
KR       0.249510
JPM      0.378733
BAC      0.310238
HSBC     0.399365
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
         PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
46  0.000685  0.002793  0.016877  0.003225 -0.040756  0.013558  0.007602   
47  0.000918  0.003325  0.000939 -0.000928  0.008089  0.058119  0.069868   
48 -0.000579  0.000101  0

...

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.00308979 0.0527506  0.         0.06872588 0.13191776 0.04664691
 0.07488055 0.0403678  0.37731919 0.20517971]
Results for day:  15
Portfolio allocation:
 [0.00308979 0.0527506  0.         0.06872588 0.13191776 0.04664691
 0.07488055 0.0403678  0.37731919 0.20517971]
Sharpe ratio:
 -0.006593395774073725
Sortino ratio:
 PFE     -0.004138
JNJ     -0.002865
GOOGL   -0.002924
AAPL    -0.003139
COST    -0.002255
WMT     -0.002830
KR      -0.001490
JPM     -0.002262
BAC     -0.001853
HSBC    -0.002385
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
          PFE       JNJ     GOOGL      AAPL      COST       WMT        KR  \
136  0.005035  0.002262 -0.002149  0.012334  0.039992  0.002855  0.098959   
137 -0.004681 -0.018310 -0.002022 -0.018792 -0.001089 -0.000415 -0.083325   
138 -0.001612 -0.01

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


Portfolio allocation: [0.11598658 0.09121002 0.01335766 0.00223067 0.40107265 0.14287671
 0.04331924 0.09828122 0.02465927 0.06809747]
Results for day:  15
Portfolio allocation:
 [0.11598658 0.09121002 0.01335766 0.00223067 0.40107265 0.14287671
 0.04331924 0.09828122 0.02465927 0.06809747]
Sharpe ratio:
 0.9625571800797026
Sortino ratio:
 PFE      1.068860
JNJ      0.740022
GOOGL    0.755403
AAPL     0.810912
COST     0.582496
WMT      0.730987
KR       0.384884
JPM      0.584218
BAC      0.478560
HSBC     0.616044
dtype: float64
Max drawdown:
 PFE      0.249553
JNJ      0.441413
GOOGL    0.218153
AAPL     0.322357
COST     0.504158
WMT      0.270134
KR       0.487117
JPM      0.363776
BAC      0.485050
HSBC     0.260622
dtype: float64
Length of opt 10
[[0.086 0.004 0.051 0.117 0.218 0.137 0.    0.071 0.018 0.29 ]
 [0.285 0.051 0.187 0.082 0.016 0.038 0.026 0.057 0.075 0.173]
 [0.221 0.22  0.019 0.137 0.285 0.016 0.069 0.    0.024 0.001]
 [0.01  0.054 0.335 0.344 0.01  0.091 0.059 0.0

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
s_r

[0.18146564664356982, 0.24608812445209166, 0.3500117196440029]

In [ ]:
roi

[243.90784673782844, 177.4336435595198, 212.94799217251023]